In [49]:
from pymongo import MongoClient
import pandas as pd
import datetime

In [50]:
client = MongoClient()
characters = client.ck2.characters

This notebook tries to build a world tree by drawing and edge between every character in the save file with their father and mother. Running this code will generate a network with over 270,000 nodes out of a total of almost 400,000. This was taking far too long for Gephi to graph so I did not continue.

The next 3 notebooks contain the first code I wrote for extracting data from the save file. I wild manually copy and paste out the dynasty data from both files, the character data and the title data and save them in seperate files.

## Get Parent/Child Edges

In [72]:
pipeline = [
    {
        "$unwind" : "$parents"        
    },
    {
        "$lookup" :
        {
            "from" : "dynasties",
            "localField" : "dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"
    },
    {
        "$match" : {"parents" : {"$nin" : [None]}, "$or" : [{"cul" : "irish"}, {"dynasty.culture" : "irish"}]}
    },
    {
        "$project" : {"_id" : 1, "parents" : 1}
    }
]

In [73]:
relation_df = pd.DataFrame(list(characters.aggregate(pipeline)))

## Get all Characters

In [53]:
pipeline = [
    {
        "$lookup" :
        {
            "from" : "dynasties",
            "localField" : "dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"
    },
    {
        "$project" : {"_id" : 1, "name" : {"$concat" : ["$bn", " ", "$dynasty.name"]}, 
                      "culture" : {"$ifNull" : ["$cul", "$dynasty.culture"]}, 
                      "religion" : {"$ifNull" : ["$rel", "$dynasty.religion"]} }
    }
]

## Build Network

In [54]:
import networkx as nx
import matplotlib.pyplot as plt

In [55]:
chars = list(characters.aggregate(pipeline))

In [62]:
for char in chars:
    for key in list(char.keys()):
            val = char[key]
            if isinstance(val, type(None)):
                del char[key]

In [63]:
G = nx.Graph()

for char in chars: #characters.aggregate(pipeline):
    if "culture" in char and "religion" in char and "name" in char:
        G.add_node(char["_id"], name = char['name'], culture = char['culture'], religion = char['religion'])

In [65]:
relation_df = relation_df.dropna(axis=0, how='any')

for i in range(len(relation_df)):
        G.add_edge(relation_df.loc[i, "_id"], relation_df.loc[i, "parents"])

G.remove_nodes_from(nx.isolates(G)) #drop unconnected nodes       

In [ ]:
#nx.draw(G)
#plt.show()

In [67]:
nx.write_graphml(max(nx.connected_component_subgraphs(G), key=len), "ck2-World-Tree-2.graphml")